<a href="https://colab.research.google.com/github/aekanun2020/Data-Analytics-using-Python/blob/main/26OCT2023_of_from_Solution_Homework_to_Regression_ONLYScaledDepDelay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
! wget https://storage.googleapis.com/1sep/2008.csv

--2023-10-26 12:26:45--  https://storage.googleapis.com/1sep/2008.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.134.207, 173.194.212.207, 173.194.213.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.134.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689413344 (657M) [text/csv]
Saving to: ‘2008.csv’

2008.csv            100%[===================>] 657.48M   210MB/s    in 3.2s    

2023-10-26 12:26:49 (205 MB/s) - ‘2008.csv’ saved [689413344/689413344]



In [4]:
! wc -l 2008.csv

7009729 2008.csv


In [5]:
import pandas as pd
import random

f = "2008.csv"
num_lines = sum(1 for l in open(f))
size = int(num_lines / 100)

# ตั้งค่า seed สำหรับการสุ่ม
random.seed(42)  # คุณสามารถเปลี่ยนเลข 42 เป็นเลขอื่น ๆ ได้

skip_idx = random.sample(range(1, num_lines), num_lines - size)
raw_pd = pd.read_csv(f, skiprows=skip_idx)


In [6]:
features_pd = raw_pd[['DepDelay','ArrDelay']]

In [7]:
final_pd = features_pd.dropna()

In [8]:
import pandas as pd

# สมมติว่าเรามี DataFrame ชื่อ final_pd
correlation = final_pd['DepDelay'].corr(final_pd['ArrDelay'])

print(f"ค่าสหสัมพันธ์ระหว่าง DepDelay และ ArrDelay คือ: {correlation:.4f}")


ค่าสหสัมพันธ์ระหว่าง DepDelay และ ArrDelay คือ: 0.9304


In [9]:
final_pd.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
DepDelay,68524.0,9.875970,34.800186,-59.0,-4.0,-1.0,8.0,1153.0
ArrDelay,68524.0,8.020927,38.194390,-81.0,-10.0,-2.0,11.0,1312.0


In [10]:
final_pd.count()

DepDelay    68524
ArrDelay    68524
dtype: int64

In [11]:
import pandas as pd
from seaborn import load_dataset
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

In [12]:
train_df = final_pd.sample(frac=0.8,random_state=123)

In [13]:
test_df = final_pd.drop(train_df.index)

In [14]:
train_df.count()

DepDelay    54819
ArrDelay    54819
dtype: int64

In [15]:
test_df.count()

DepDelay    13705
ArrDelay    13705
dtype: int64

In [16]:
train_labels = train_df.filter(['ArrDelay'])
train_df = train_df.loc[:, train_df.columns != 'ArrDelay']

In [17]:
type(train_labels)

pandas.core.frame.DataFrame

In [18]:
test_labels = test_df.filter(['ArrDelay'])
test_df = test_df.loc[:, test_df.columns != 'ArrDelay']

In [19]:
X_train = train_df
X_test = test_df
y_train = train_labels
y_test = test_labels

In [20]:
categorical = list(X_train.select_dtypes('object').columns)
print(f"Categorical columns are: {categorical}")

numerical = list(X_train.select_dtypes('number').columns)
print(f"Numerical columns are: {numerical}")

Categorical columns are: []
Numerical columns are: ['DepDelay']


In [21]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz, DecisionTreeRegressor
from sklearn import tree
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [22]:
# Define custom transformer
class ColumnSelector(BaseEstimator, TransformerMixin):
    """Select only specified columns."""
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.columns]

# Define categorical pipeline
#cat_pipe = Pipeline([('selector', ColumnSelector(categorical)),
#                     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
#                     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))])

# Define numerical pipeline
num_pipe = Pipeline([('selector', ColumnSelector(numerical)),
                     #('imputer', SimpleImputer(strategy='median')),
                     ('scaler', MinMaxScaler())
                    ])

# Combine categorical and numerical pipeline
preprocessor = FeatureUnion(transformer_list=[
    #('cat', cat_pipe),
                                              ('num', num_pipe)])

# Model Training
pipe = Pipeline(steps=[('preprocessor', preprocessor),
              ('model', DecisionTreeRegressor(max_depth=None))])
pipeline_model = pipe.fit(X_train, y_train)

In [23]:
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

predictions = pipeline_model.predict(X_test)
print("\t RMSE:", mean_squared_error(y_test, predictions, squared=False))
print("\t R2 score:", r2_score(y_test, predictions))

	 RMSE: 14.614705275246102
	 R2 score: 0.8743034941647341
